In [2]:
from pymongo import MongoClient, TEXT, GEOSPHERE
from pprint import pprint

client = MongoClient()
db = client.geo
restaurants = db.restaurants
states = db.states

In [3]:
restaurants.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'geo.restaurants', 'v': 2},
 'address.coord_2dsphere': {'2dsphereIndexVersion': 3,
  'key': [('address.coord', '2dsphere')],
  'ns': 'geo.restaurants',
  'v': 2},
 'address.zipcode_1': {'key': [('address.zipcode', 1)],
  'ns': 'geo.restaurants',
  'v': 2},
 'grades.grade_1': {'key': [('grades.grade', 1)],
  'ns': 'geo.restaurants',
  'v': 2},
 'name_text': {'default_language': 'english',
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'language_override': 'language',
  'ns': 'geo.restaurants',
  'textIndexVersion': 3,
  'v': 2,
  'weights': SON([('name', 1)])}}

# Exercise 1-1

In [4]:
result = restaurants.find({
    '$text': {'$search': 'Kimchi'}}, {
    '_id': 0,
    'address': 1,
    'name':1
})
list(result)

[{'address': {'building': '478',
   'coord': [-73.9978033, 40.674716],
   'street': 'Smith Street',
   'zipcode': '11231'},
  'name': 'Kimchi Grill'},
 {'address': {'building': '766',
   'coord': [-73.9634926, 40.6750939],
   'street': 'Washington Avenue',
   'zipcode': '11238'},
  'name': 'Kimchi Grill'},
 {'address': {'building': '18-20',
   'coord': [-73.97853599999999, 40.757446],
   'street': 'West 48 Street',
   'zipcode': '10036'},
  'name': 'New York Kimchi'}]

# Exercise 1-2

In [5]:
nolbu_coord = restaurants.find_one({
    '$text': {
        '$search': 'Nolbu Restaurant'
    }}, {
    '_id': 0,
    'address': 1
})
print('--- nolbu ---')
pprint(nolbu_coord)


--- nolbu ---
{'address': {'building': '164-25',
             'coord': [-73.8013625, 40.7605922],
             'street': 'Northern Boulevard',
             'zipcode': '11358'}}


In [6]:
name = states.find_one({
    'loc': {
        '$geoIntersects': {
            '$geometry': {'type': 'Point', 'coordinates': nolbu_coord['address']['coord']}}}})['name']


print('--- result ---')
print(name)

--- result ---
New York


# Exercise 1-3

In [10]:
r_obj = restaurants.find_one({
    'cuisine': 'Korean',
    '$text': {
        '$search': 'ramen -izakaya'}})
pprint(r_obj)

{'_id': ObjectId('5c8626739f44c9f676802de4'),
 'address': {'building': '95',
             'coord': [-73.9923524, 40.7238111],
             'street': 'East Houston Street',
             'zipcode': '10002'},
 'borough': 'Manhattan',
 'cuisine': 'Korean',
 'grades': [{'date': datetime.datetime(2014, 4, 28, 0, 0),
             'grade': 'A',
             'score': 12}],
 'name': 'Yuji Ramen',
 'restaurant_id': '50008658'}


In [11]:
geo_obj = {
    'type': 'Point',
    'coordinates': r_obj['address']['coord']
}

result = restaurants.find({
    'cuisine': 'Japanese',
    'address.coord': {
        '$near': {
            '$geometry': geo_obj,
            '$maxDistance': 150
        }
    },
    'name': {
        '$ne': r_obj['name']
    }
})

pprint(list(result))

[{'_id': ObjectId('5c8626739f44c9f6768035da'),
  'address': {'building': '8',
              'coord': [-73.9916305, 40.7247106],
              'street': 'Extra Pl',
              'zipcode': '10003'},
  'borough': 'Manhattan',
  'cuisine': 'Japanese',
  'grades': [{'date': datetime.datetime(2015, 1, 20, 0, 0),
              'grade': 'Not Yet Graded',
              'score': 21}],
  'name': 'Ko Ep, Llc',
  'restaurant_id': '50015854'}]


In [12]:
client.close()